# Crawl a website 

We will use `DPK-Connector` module

References
- https://github.com/IBM/data-prep-kit/tree/dev/data-connector-lib


## Step-1: Config

In [1]:
from my_config import MY_CONFIG

## Step-2: Setup Directories

In [2]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DIR ):
    shutil.os.makedirs(MY_CONFIG.INPUT_DIR, exist_ok=True)

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_DIR, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


# Step-3: Crawl the website

We will use `dpk-connector` utility to download a some HTML pages from a site 

In [3]:
from dpk_connector import crawl, shutdown
import nest_asyncio
import os
from my_utils import get_mime_type, get_filename_from_url
from dpk_connector.core.utils import validate_url

# Use nest_asyncio to enable a nested event loop run for the crawler inside the Jupyter notebook
nest_asyncio.apply()

# Initialize counter
retrieved_pages = 0
saved_pages = 0

# Define a callback function to be executed at the retrieval of each page during a crawl
def on_downloaded(url: str, body: bytes, headers: dict) -> None:
    """
    Callback function called when a page has been downloaded.
    You have access to the request URL, response body and headers.
    """
    global retrieved_pages, saved_pages
    retrieved_pages+=1
    
    print (f'on_downloaded (url={url})')

    if saved_pages < MY_CONFIG.CRAWL_MAX_DOWNLOADS:
        print(f"Visited url: {url}")

    # Get mime_type of retrieved page
    mime_type = get_mime_type(body)
    print (f'mime_type={mime_type}')
    
    # Save the page if it is a PDF to only download research papers
    if MY_CONFIG.CRAWL_MIME_TYPE  in mime_type.lower():
        print ('hi')
        filename = get_filename_from_url(url)
        local_file_path = os.path.join(MY_CONFIG.INPUT_DIR, filename)
        print (local_file_path)
        
        with open(local_file_path, 'wb') as f:
            f.write(body)
            
        if saved_pages < MY_CONFIG.CRAWL_MAX_DOWNLOADS :
            print(f"Saved contents of url: {url}")
        saved_pages+=1
        
# Define a user agent to provide information about the client making the request
# user_agent = "dpk-connector"
user_agent = "Mozilla/5.0 (X11; Linux i686; rv:125.0) Gecko/20100101 Firefox/125.0"


def on_downloaded2(url: str, body: bytes, headers: dict) -> None:
    # print(f"url: {url}, headers: {headers}, body: {body[:64]}")
    print(f"url: {url}")

async def run_my_crawl():
    crawl([MY_CONFIG.CRAWL_URL_BASE], 
          on_downloaded2,  
          user_agent=user_agent, 
          depth_limit = MY_CONFIG.CRAWL_MAX_DEPTH, 
          subdomain_focus=True,
        #   path_focus = True, 
          download_limit = MY_CONFIG.CRAWL_MAX_DOWNLOADS)
    return "Crawl is done"

# Now run the configured crawl
await run_my_crawl()

url: https://thealliance.ai/
url: https://thealliance.ai/core-projects/ntia_request
url: https://thealliance.ai/contact
url: https://thealliance.ai/blog/open-source-ai-demo-night-sf-2024
url: https://thealliance.ai/core-projects/trusted-evals
url: https://thealliance.ai/core-projects/sb1047
url: https://thealliance.ai/focus-areas/hardware-enablement
url: https://thealliance.ai/focus-areas/applications-and-tools
url: https://thealliance.ai/focus-areas/trust-and-safety
url: https://thealliance.ai/community
url: https://thealliance.ai/focus-areas/advocacy
url: https://thealliance.ai/focus-areas/skills-education
url: https://thealliance.ai/contribute
url: https://thealliance.ai/become-a-collaborator
url: https://thealliance.ai/focus-areas/foundation-models-datasets
url: https://thealliance.ai/aia-members
url: https://thealliance.ai/our-work
url: https://thealliance.ai/about-aia
url: https://thealliance.ai/core-projects/the-living-guide-to-applying-ai
url: https://thealliance.ai/core-projec

'Crawl is done'